In [1]:
# ruff: noqa: F401, ANN201

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import altair as alt
import duckdb
import hvplot.polars
import numpy as np
import polars as pl

In [5]:
from tsdb_benchmarks.settings import SETTINGS, DatabaseName, Operation, SuiteName

In [6]:
db = duckdb.connect(SETTINGS.results_directory / "results.db")

In [7]:
db.execute("select * from benchmark where finished_at is not null and deleted_at is null order by started_at").pl()

id,suite,db,operation,started_at,finished_at,deleted_at,notes
i32,str,str,str,datetime[μs],datetime[μs],datetime[μs],str
1,"""rtabench""","""duckdb""","""populate""",2025-07-18 09:30:03.695574,2025-07-18 09:31:03.239055,null,null
2,"""rtabench""","""duckdb""","""run""",2025-07-18 09:31:14.367992,2025-07-18 09:31:19.951452,null,null
3,"""rtabench""","""duckdb""","""run""",2025-07-18 10:06:50.229108,2025-07-18 10:07:38.298605,null,null


In [13]:
def get_latest_id(suite: SuiteName, db_name: DatabaseName, operation: Operation) -> int:
    return db.execute(
        """
        select id from benchmark where suite = (?) and db = (?) and operation = (?)
            and finished_at is not null order by finished_at desc limit 1
        """,
        [suite, db_name, operation],
    ).fetchone()[0]


def plot_queries(suite: SuiteName, db_name: DatabaseName, operation: Operation):
    df = db.execute(
        """
        with filtered as (
            select *
            from event
            where benchmark_id = $id
            and name like 'query_%'
        ),
        starts as (
            select name, time as start_time
            from filtered
            where type = 'start'
        ),
        ends as (
            select name, time as end_time
            from filtered
            where type = 'end'
        )
        select
            substring(starts.name, 7, 4) as name,
            starts.start_time,
            ends.end_time,
            epoch(ends.end_time - starts.start_time) as duration
        from starts
        join ends using(name)
        order by starts.start_time
        """,
        {"id": get_latest_id(suite, db_name, operation)},
    ).pl()

    # Aggregate: mean and std deviation
    agg_df = (
        df.group_by("name")
        .agg([pl.col("duration").mean().alias("avg_duration"), pl.col("duration").std().alias("std_duration")])
        .sort("name")
    )

    # Convert to pandas for Altair (if needed)
    pdf = agg_df.to_pandas()

    title = f"{suite}/{db_name}/{operation}"
    chart = (
        alt.Chart(pdf)
        .mark_bar()
        .encode(
            x=alt.X("name:N", title="Query Name"),
            y=alt.Y("avg_duration:Q", title="Avg Duration (s)"),
        )
        .properties(title=title)
    )

    error_bars = (
        alt.Chart(pdf)
        .mark_errorbar(thickness=4, color="red", extent="stdev")
        .encode(x="name:N", y="avg_duration:Q", yError="std_duration:Q")
    )

    return chart + error_bars


plot_queries("rtabench", "duckdb", "run")

alt.LayerChart(...)

In [14]:
def plot_metrics(suite: SuiteName, db_name: DatabaseName, operation: Operation):
    df = db.execute(
        """
        SELECT
            epoch(time - (SELECT min(time) FROM metric WHERE benchmark_id = $id)) AS time,
            mem_mb, disk_mb, cpu_percent
        FROM metric
        WHERE benchmark_id = $id
        ORDER BY time
        """,
        {"id": get_latest_id(suite, db_name, operation)},
    ).pl()

    base = alt.Chart(df).encode(x="time:N")

    return (
        alt.layer(
            *[
                base.mark_line(color=c).encode(alt.Y(n).axis(titleColor=c))
                for n, c in {"mem_mb": "red", "disk_mb": "green", "cpu_percent": "orange"}.items()
            ]
        )
        .resolve_scale(y="independent")
        .properties(width=1000)
        .interactive()
    )


plot_metrics("rtabench", "duckdb", "run")

alt.LayerChart(...)